# directory structure exploration


In [3]:
bucket="gs://dsgt-clef-erisk-2024"
! gcloud storage ls {bucket}

gs://dsgt-clef-erisk-2024/task1/
gs://dsgt-clef-erisk-2024/task2/


In [4]:
! gcloud storage ls {bucket}/task1

gs://dsgt-clef-erisk-2024/task1/dataset/
gs://dsgt-clef-erisk-2024/task1/training/


In [18]:
! gcloud storage du --readable-sizes {bucket}/task1/dataset | head -n 5
! gcloud storage du --readable-sizes {bucket}/task1/dataset | tail -n 5
! gcloud storage ls {bucket}/task1/dataset | wc -l

3.54kiB      gs://dsgt-clef-erisk-2024/task1/dataset/s_0.trec
10.73MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_1.trec
10.65MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_10.trec
10.95MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_100.trec
10.77MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_101.trec
11.15MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_98.trec
10.62MiB     gs://dsgt-clef-erisk-2024/task1/dataset/s_99.trec
839.27MiB    gs://dsgt-clef-erisk-2024/task1/dataset/t1_test_collection_2024.zip
2.53GiB      gs://dsgt-clef-erisk-2024/task1/dataset/test.parquet
9.35GiB      gs://dsgt-clef-erisk-2024/task1/dataset/
555


In [10]:
! gcloud storage du --readable-sizes {bucket}/task1/training

gs://dsgt-clef-erisk-2024/task1/training/training.parquet
gs://dsgt-clef-erisk-2024/task1/training/t1_training/


In [21]:
! gcloud storage du --readable-sizes "{bucket}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/" | head -n 5
! gcloud storage du --readable-sizes "{bucket}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/" | tail -n 5
! gcloud storage du "{bucket}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/" | wc -l

406.81kiB    gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_qrels_majority_2.csv
406.81kiB    gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_rels_consenso.csv
22.86kiB     gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_0.trec
44.65kiB     gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_1.trec
200.45kiB    gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_10.trec
305.75kiB    gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_997.trec
2.09kiB      gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_998.trec
190.46kiB    gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/new_data/s_999.trec
745.75MiB    gs://dsgt-clef-erisk-2024/task1/training/t1_t